In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier

Etiquetamos los nombres de las características en el dataset.

In [2]:
featuresNames = ['MeanIntegratedProfile',
                'StdIntegratedProfile',
                'ExcessKurtosisIntegratedProfile',
                'SkewnessIntegratedProfile',
                'MeanDMSNRCurve',
                'StdDMSNRCurve',
                'ExcessKurtosisDMSNRCurve',
                'SkewnessDMSNRCurve',
                'Class']

Cargamos la informacion del dataset HTRU2.

In [3]:
data = pd.read_csv('../HTRU_2.csv',
                   header = None, 
                   names = featuresNames)

Separamos la variable categórica del dataset.

In [4]:
x = data.drop('Class', axis = 1, inplace = False)
y = data['Class']

Se divide el dataset en dos conjuntos de datos: entrenamiento y pruebas. El conjunto de entrenamiento estará compuesto por el 90% del dataset y pruebas del 10%.

El dataset HTRU2 es un conjunto de datos desbalanceado, por cada púlsar identificado, diez no lo son. Para que esta misma proporción se refleje en los conjuntos de entrenamiento y pruebas se realizará una división estratificada.

In [5]:
y.value_counts()

0    16259
1     1639
Name: Class, dtype: int64

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x,
                                                    y,
                                                    test_size = 0.1,
                                                    random_state = 0,
                                                    stratify = y)

Ahora ambos conjuntos de datos tienen la misma proporción con respecto a la variable categórica.

In [7]:
print(y_train.value_counts())
print(y_test.value_counts())

0    14633
1     1475
Name: Class, dtype: int64
0    1626
1     164
Name: Class, dtype: int64


Como mencionamos anteriormente, el dataset HTRU2 es conjunto de datos desbalanceado. Para balancear utilizaremos una técnica de oversampling llamada SMOTE, donde se genera nuevas instancias de la clase minoritaria interpolando los valores de las instancias minoritarias más cercanas a una dada. El nuevo ratio será de 1:5, esto quiere decir, que por cada púlsar, cinco no lo serán en el conjunto de entrenamiento.

In [8]:
smote = SMOTE(random_state = 0,
             ratio = 0.2)
x_train_smote, y_train_smote = smote.fit_sample(x_train, y_train.ravel())

Ahora podemos observar que el conjunto de datos se encuentra balanceado.

In [9]:
np.bincount(y_train_smote)

array([14633,  2926])

Para encontrar los mejores hiper-parámetros que se ajusten al modelo utilizaremos la técnica de búsqueda en grilla. 

Para determinar los mejores valores de criterion, max_depth, min_samples_split y min_samples_leaf será necesario probar el modelo con diferentes valores.

En cada iteración se realizará validación cruzada con k = 5.

La métrica de evaluación será f1.

In [10]:
param_grid = {  
    'criterion': ['gini', 'entropy'],
    'max_depth': [1, 3, 5, 7],
    'min_samples_split': [2, 5, 7, 9],
    'min_samples_leaf': [1, 3, 5, 7]
}

grid_search = GridSearchCV(estimator = DecisionTreeClassifier(),
                           param_grid = param_grid,
                           scoring = 'f1',
                           cv = 5)

grid_search.fit(x_train_smote, y_train_smote)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [1, 3, 5, 7], 'min_samples_split': [2, 5, 7, 9], 'min_samples_leaf': [1, 3, 5, 7]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [11]:
print('Tuneando los hiper-parámetros para f1')
print()

print('Los mejores hiper-parámetros encontrados con validación cruzada:')
print()
print(grid_search.best_params_)
print()
print('Puntajes de la métrica f1 en el conjunto de validación:')
print()
means = grid_search.cv_results_['mean_test_score']
stds = grid_search.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, grid_search.cv_results_['params']):
    print('%0.4f (+/-%0.04f) for %r'
          % (mean, std * 2, params))

Tuneando los hiper-parámetros para f1

Los mejores hiper-parámetros encontrados con validación cruzada:

{'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 9}

Puntajes de la métrica f1 en el conjunto de validación:

0.8897 (+/-0.0115) for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 2}
0.8897 (+/-0.0115) for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 5}
0.8897 (+/-0.0115) for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 7}
0.8897 (+/-0.0115) for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 1, 'min_samples_split': 9}
0.8897 (+/-0.0115) for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 3, 'min_samples_split': 2}
0.8897 (+/-0.0115) for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 3, 'min_samples_split': 5}
0.8897 (+/-0.0115) for {'criterion': 'gini', 'max_depth': 1, 'min_samples_leaf': 3, 'min_samples_split':

La búsqueda en grilla nos devuelve el modelo con los mejores hiper-parámetros.

Finalmente, vamos a predecir los resultados del modelo con los conjuntos de datos de entrenamiento y pruebas.

In [12]:
decision_tree_model = grid_search

decision_tree_model_train_prediction = decision_tree_model.predict(x_train_smote)
decision_tree_model_test_prediction = decision_tree_model.predict(x_test)

In [14]:
print('Árbol de decisión')
print('\t-Mejores hiper-parámetros: ' + str(decision_tree_model.best_params_))
print('\t-Puntaje f1 en entrenamiento: %.4f' % f1_score(y_train_smote, decision_tree_model_train_prediction, average = 'binary'))
print('\t-Puntaje f1 en pruebas: %.4f' % f1_score(y_test, decision_tree_model_test_prediction, average = 'binary'))

Árbol de decisión
	-Mejores hiper-parámetros: {'criterion': 'gini', 'max_depth': 7, 'min_samples_leaf': 5, 'min_samples_split': 9}
	-Puntaje f1 en entrenamiento: 0.9286
	-Puntaje f1 en pruebas: 0.8862
